# Example of how to use LLMs-powered agents.

The examples show how to use LLMs-powered agents. There are many frameworks that allow one to architect systems that solve a problem using agents. Among the most popular ones are [CrewAI](https://docs.crewai.com/) and [AutoGen](https://microsoft.github.io/autogen/docs/Getting-Started/). [LangChain](https://python.langchain.com/v0.1/docs/modules/agents/) also provides a simple framework to use agents, and that is what we will use in this notebook.

This agents can use local or online models. For instructions on how to install and use local models see the `using_llms` notebook.

In [ ]:
# read secrets ------------------------------------------------
import json
import os

# you define your on secrets.json file with the following structure
# {
#     "openai": "your-openai-api-key",
#     "groq": "your-groq-api-key"
# }

with open("./secrets.json") as f:
    secrets = json.load(f)

os.environ["OPENAI_API_KEY"] = secrets["openai"]
os.environ["GROQ_API_KEY"] = secrets["groq"]
os.environ["SERPER_API_KEY"] = secrets["serper_key"]


## Example of an agent using an agent that uses a tool

In [ ]:
from langchain.agents import (
    AgentExecutor,
    load_tools,
    create_tool_calling_agent
)
from langchain_community.llms import Ollama
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(model="gpt-4o")
# llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
# llm = Ollama(model="phi3", temperature=0, keep_alive='10m')

# using https://serper.dev/ for search - go there and get a key
tools = load_tools(["google-serper"], llm=llm)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(tools=tools, llm=llm, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
# Test the agent
agent_executor.invoke({"input": "What is the weather in Luxemburg?"})


In [ ]:
# Now put things together agents, tools, and chains

from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

search_prompt = ChatPromptTemplate.from_template("{question}")
translate_prompt = ChatPromptTemplate.from_template(
    "translate to {language} the following: {text}"
)

# model = ChatOpenAI()
# model = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
model = Ollama(model="phi3", temperature=0, keep_alive='10m')

serp_chain = {"input": search_prompt} | agent_executor 

translate_chain = (
    {"text": serp_chain, "language": itemgetter("language")}
    | translate_prompt
    | model
    | StrOutputParser()
)

result = translate_chain.invoke({"question": "what is the weather in luxemburg (in metric units)?", "language": "spanish"})

print(result)